In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
structure = pd.read_csv("kaggle/structure.csv")
pd_targets = pd.read_csv("kaggle/targets.csv")
df = pd.read_csv("kaggle/user_activity.csv")
df1 = pd.read_csv("kaggle/user_activity_test.csv")

In [3]:
df.head()

,user_id,action,step_id,time,step_type,step_cost
0,5920,discovered,2681,1403165936,text,0
1,5920,passed,2681,1403165936,text,0
2,5920,viewed,2681,1403165936,text,0
3,5920,passed,2682,1403165946,text,0
4,5920,viewed,2682,1403165946,text,0


In [4]:
np.where('code' == np.append(df.action.unique(), df.step_type.unique()))[0][0]

6

In [5]:
X = np.zeros((df.user_id.unique().size, 6103, 6), dtype="int32")

In [6]:
indexer = np.append(df.action.unique(), df.step_type.unique())

t = df.groupby("user_id")
i = 0
for id_ in tqdm_notebook(df.user_id.unique()):
    tx = t.get_group(id_)
    z = np.zeros((6103, 6))
    
    k = 0
    for _, row in tx.iterrows():
        l = row.values
        l[1] = np.where(l[1] == indexer)[0][0]
        l[4] = np.where(l[4] == indexer)[0][0]
        z[k] = l
        k += 1
    
    X[i] = z
    i += 1

KeyboardInterrupt: 

In [7]:
X = np.load("X.npy")

In [ ]:
np.save("X", X)

In [8]:
import theano  # нейронки
import numpy as np  # наука
import lasagne  # готовые нейронки
import theano.tensor as T

In [31]:
input_X = T.tensor3("X", dtype="int32")
input_shape = [None, 6103, 6]
target_y = T.vector("target Y", dtype='int32')

#АРХИТЕКТУРА нейронки:
# #входной слой (вспомогательный)
input_layer = lasagne.layers.InputLayer(shape=input_shape, input_var=input_X)

#hidden layers
dense_0 = lasagne.layers.DenseLayer(input_layer, 40, nonlinearity=lasagne.nonlinearities.sigmoid)
dense_output = lasagne.layers.DenseLayer(dense_0,
                                        num_units = 1,
                                        nonlinearity = lasagne.nonlinearities.softmax,
                                        name='output')
#/АРХИТЕКТУРА нейронки

#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(dense_output)

#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(dense_output)

#функция ошибки - средняя кроссэнтропия
loss = lasagne.objectives.binary_crossentropy(y_predicted, target_y).mean()

#сразу посчитать словарь обновлённых значений с шагом по градиенту, как раньше
updates_sgd = lasagne.updates.rmsprop(loss, all_weights,learning_rate=0.01)

#функция, которая обучает сеть на 1 шаг и возвращащет значение функции потерь и точности
train_fun = theano.function([input_X,target_y], loss, updates=updates_sgd, allow_input_downcast=True)

# вспомогательная функция, которая возвращает список мини-батчей для обучения нейронки

#на вход
# X - тензор из картинок размером (много, 1, 28, 28), например - X_train
# y - вектор из чиселок - ответов для каждой картинки из X; например - Y_train
#batch_size - одно число - желаемый размер группы

#что нужно сделать
# 1) перемешать данные
# - важно перемешать X и y одним и тем же образом, чтобы сохранить соответствие картинки ответу на неё
# 3) побить данные на подгруппы так, чтобы в каждой подгруппе было batch_size картинок и ответов
# - если число картинок не делится на batch_size, одну подгруппу можно вернуть другого размера
# 4) вернуть список (или итератор) пар:
# - (подгруппа картинок, ответы из y на эту подгруппу)

def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt] 

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, pd_targets.passed.ravel())

In [33]:
#ОБУЧЕНИЕ И ВАЛИДАЦИЯ
import time

num_epochs = 10 #количество проходов по данным

batch_size = 100 #размер мини-батча

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, Y_train, batch_size):
        inputs, targets = batch
        train_err_batch = train_fun(inputs, targets)
        train_err += train_err_batch
        train_batches += 1
        
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))

Epoch 1 of 10 took 13.454s
  training loss (in-iteration):		nan
Epoch 2 of 10 took 16.052s
  training loss (in-iteration):		nan
Epoch 3 of 10 took 12.577s
  training loss (in-iteration):		nan
Epoch 4 of 10 took 10.319s
  training loss (in-iteration):		nan
Epoch 5 of 10 took 12.694s
  training loss (in-iteration):		nan
Epoch 6 of 10 took 14.291s
  training loss (in-iteration):		nan
Epoch 7 of 10 took 11.707s
  training loss (in-iteration):		nan
Epoch 8 of 10 took 9.829s
  training loss (in-iteration):		nan
Epoch 9 of 10 took 9.547s
  training loss (in-iteration):		nan
Epoch 10 of 10 took 11.034s
  training loss (in-iteration):		nan


In [34]:
predict_net = theano.compile.function([input_X], lasagne.layers.get_output(dense_output))

In [35]:
predict_net(np.array([X[4]]))

array([[ nan]])